In [1]:
from qutip import *
from numpy import savetxt, real, pi

In [32]:
s0 = Qobj([[1, 0], [0, 0]])
s1 = Qobj([[0, 0], [0, 1]])
I = qeye(2); a = destroy(2); ad = create(2)
rho111 = tensor(s1,s1,s1,s0,s0,s0)
z1 = tensor(sigmaz(),I,I,I,I,I)
z2 = tensor(I,sigmaz(),I,I,I,I)
z3 = tensor(I,I,sigmaz(),I,I,I)
x1 = tensor(sigmax(),I,I,I,I,I)
x2 = tensor(I,sigmax(),I,I,I,I)
x3 = tensor(I,I,sigmax(),I,I,I)
a1 = tensor(I,I,I,a,I,I); a1d = tensor(I,I,I,ad,I,I)
a2 = tensor(I,I,I,I,a,I); a2d = tensor(I,I,I,I,ad,I)
a3 = tensor(I,I,I,I,I,a); a3d = tensor(I,I,I,I,I,ad)
HP = -0.02*(z1*z2 + z1*z3 + z2*z3)

In [31]:
savetxt('doink.txt',real(rho111.full()))

In [33]:
def lindbladME(cp, cs, rho, H):
    return 2j*pi*(rho*H - H*rho) + cp*(x1*rho*x1 - rho) + cs*(a1*rho*a1d - 0.5*(a1d*a1*rho + rho*a1d*a1)) + cp*(x2*rho*x2 - rho) + cs*(a2*rho*a2d - 0.5*(a2d*a2*rho + rho*a2d*a2)) + cp*(x3*rho*x3 - rho) + cs*(a3*rho*a3d - 0.5*(a3d*a3*rho + rho*a3d*a3))

In [36]:
mat = lindbladME(1e-3/20, 0.03, rho111, HP)
(rho111*mat.dag()).tr()

-0.00015000000000000001

In [24]:
1e-3/20

5e-05

In [ ]:
 inline void basic_funcs::lindbladME(float cp, float cs, MatrixXcd& rho, MatrixXcd& H, MatrixXcd& output) {
    output = 2.0*IM*PI*(rho*H - H*rho) 
             + cp*(X1*rho*X1 - 0.5*(X1*rho + rho*X1)) + cs*(a1*rho*a1d - 0.5*(a1d*a1*rho + rho*a1d*a1))
             + cp*(X2*rho*X2 - 0.5*(X2*rho + rho*X2)) + cs*(a2*rho*a2d - 0.5*(a2d*a2*rho + rho*a2d*a2))
             + cp*(X3*rho*X3 - 0.5*(X3*rho + rho*X3)) + cs*(a3*rho*a3d - 0.5*(a3d*a3*rho + rho*a3d*a3));
    return;
}

inline void basic_funcs::lindbladRK4(float col1, float col2, float step, MatrixXcd& rho, MatrixXcd& H, MatrixXcd& output) {
    MatrixXcd t1, t2, t3, k1, k2, k3, k4;
    lindbladME(col1, col2, rho, H, k1);
    t1 = rho + 0.5*step*k1; lindbladME(col1, col2, t1, H, k2);
    t2 = rho + 0.5*step*k2; lindbladME(col1, col2, t2, H, k3);
    t3 = rho + step*k3; lindbladME(col1, col2, t3, H, k4);
    output = rho + step*(k1 + 2*k2 + 2*k3 + k4)/6.0;
    return;
}